In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
class Arm:
    def __init__(self, p):
        self.p = p

    def pull(self):
        return np.random.binomial(1, self.p)

In [ ]:

class MultiBandit:
    def __init__(self, probs=[0.1, 0.2, 0.7, 0.5]):
        self.__arms = [Arm(p) for p in probs]
        self.__regret = 0
        self.__maxp = max(probs)

    def num_arms(self):
        return len(self.__arms)

    def pull(self, arm_num):
        reward = self.__arms[arm_num].pull()
        self.__regret += self.__maxp - self.__arms[arm_num].p
        return reward

    def regret(self):
        return self.__regret

In [ ]:
class EpsilonGreedyAlgorithm:
    def __init__(self, num_arms, horizon, epsilon):
        self.num_arms = num_arms
        self.horizon = horizon
        self.epsilon = epsilon
        self.timestep = 0
        self.arm_pulls = np.zeros(num_arms)
        self.arm_rewards = np.zeros(num_arms)
        self.regrets = np.zeros(horizon)

    def give_best_arm(self):
        return np.argmax(self.arm_rewards / (self.arm_pulls + 1e-6))

    def select_arm(self):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.num_arms)
        else:
            return np.argmax(self.arm_rewards / (self.arm_pulls + 1e-6))

    def run_algorithm(self, bandit):
        for _ in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            self.arm_pulls[arm_to_pull] += 1
            self.arm_rewards[arm_to_pull] += reward
            self.timestep += 1
            self.regrets[_] = bandit.regret()

    def plot(self):
        plt.plot(np.cumsum(self.regrets), label='Epsilon-Greedy')
        plt.xlabel('Timesteps')
        plt.ylabel('Total Regret')
        plt.title('Total Regret vs Timesteps')


In [ ]:
class UCBEpsilonAlgorithm:
    def __init__(self, num_arms, horizon, c):
        self.num_arms = num_arms
        self.horizon = horizon
        self.c = c
        self.timestep = 0
        self.arm_pulls = np.zeros(num_arms)
        self.arm_rewards = np.zeros(num_arms)
        self.regrets = np.zeros(horizon)

    def give_best_arm(self):
        return np.argmax(self.arm_rewards / (self.arm_pulls + 1e-6))

    def select_arm(self):
        if self.timestep < self.num_arms:
            return self.timestep
        else:
            ucb_values = self.arm_rewards / (self.arm_pulls + 1e-6) + self.c * np.sqrt(np.log(self.timestep) / (self.arm_pulls + 1e-6))
            return np.argmax(ucb_values)

    def run_algorithm(self, bandit):
        for _ in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            self.arm_pulls[arm_to_pull] += 1
            self.arm_rewards[arm_to_pull] += reward
            self.timestep += 1
            self.regrets[_] = bandit.regret()

    def plot(self):
        plt.plot(np.cumsum(self.regrets), label='UCB Epsilon')
        plt.xlabel('Timesteps')
        plt.ylabel('Total Regret')
        plt.title('Total Regret vs Timesteps')


In [ ]:
class ThompsonSamplingAlgorithm:
    def __init__(self, num_arms, horizon):
        self.num_arms = num_arms
        self.horizon = horizon
        self.timestep = 0
        self.arm_pulls = np.zeros(num_arms)
        self.arm_rewards = np.zeros(num_arms)
        self.regrets = np.zeros(horizon)

    def give_best_arm(self):
        return np.argmax(self.arm_rewards / (self.arm_pulls + 1e-6))

    def select_arm(self):
        sampled_theta = np.random.beta(self.arm_rewards + 1, self.arm_pulls - self.arm_rewards + 1)
        return np.argmax(sampled_theta)

    def run_algorithm(self, bandit):
        for _ in range(self.horizon):
            arm_to_pull = self.select_arm()
            reward = bandit.pull(arm_to_pull)
            self.arm_pulls[arm_to_pull] += 1
            self.arm_rewards[arm_to_pull] += reward
            self.timestep += 1
            self.regrets[_] = bandit.regret()

    def plot(self):
        plt.plot(np.cumsum(self.regrets), label='Thompson Sampling')
        plt.xlabel('Timesteps')
        plt.ylabel('Total Regret')
        plt.title('Total Regret vs Timesteps')


In [ ]:
# Define a MultiBandit instance
bandit = MultiBandit()

# Set the horizon size
H = 100

# Create instances of each algorithm
epsilon_greedy_algorithm = EpsilonGreedyAlgorithm(num_arms=bandit.num_arms(), horizon=H, epsilon=0.2)
ucb_epsilon_algorithm = UCBEpsilonAlgorithm(num_arms=bandit.num_arms(), horizon=H, c=2.0)
thompson_sampling_algorithm = ThompsonSamplingAlgorithm(num_arms=bandit.num_arms(), horizon=H)

# Run each algorithm
epsilon_greedy_algorithm.run_algorithm(bandit)
ucb_epsilon_algorithm.run_algorithm(bandit)
thompson_sampling_algorithm.run_algorithm(bandit)

# Display total regret for each algorithm
print(f"Epsilon-Greedy Algorithm - Total Regret after {H} timesteps: {bandit.regret()} with assumed best arm {epsilon_greedy_algorithm.give_best_arm()}")
print(f"UCB Epsilon Algorithm - Total Regret after {H} timesteps: {bandit.regret()} with assumed best arm {ucb_epsilon_algorithm.give_best_arm()}")
print(f"Thompson Sampling Algorithm - Total Regret after {H} timesteps: {bandit.regret()} with assumed best arm {thompson_sampling_algorithm.give_best_arm()}")

# Plot the regrets for each algorithm
plt.legend()
plt.show()
